In [1]:
from pyflink.table import (
    EnvironmentSettings,
    TableEnvironment,
    TableDescriptor,
    Schema,
    DataTypes,
    FormatDescriptor,
    EnvironmentSettings
)
from pyflink.table.window import Tumble,Slide, Over
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import expressions as expr
from pyflink.table.expressions import col,call, lit
from pyflink.table.udf import udf

In [2]:
env_settings = EnvironmentSettings.in_streaming_mode()
t_env = TableEnvironment.create(env_settings)    # Point to the SQL Kafka connector JAR
kafka_connector_jar = "file:///Users/gerardomartinez/DataEng/alpaca/flink-connectors/flink-sql-connector-kafka-3.3.0.jar"

t_env.get_config().set("pipeline.jars", kafka_connector_jar)

dockerC = "localhost:9092"

# t_env.get_config().set("table.exec.emit.early-fire.enabled", "false")
# t_env.get_config().set("table.exec.emit.allow-lateness", "0s")

In [3]:
source_ddl = f"""
        create table video_data(
            videoid STRING,
            viewcount BIGINT,
            log_time AS PROCTIME()
        ) with (
            'connector' = 'kafka',
            'topic' = 'track_video',
            'properties.bootstrap.servers' = '{dockerC}',
            'properties.group.id' = 'transaction_group',
            'scan.startup.mode' = 'earliest-offset',
            'format' = 'json',
            'json.fail-on-missing-field' = 'false',
            'json.ignore-parse-errors' = 'true'
        );
        """

t_env.execute_sql(source_ddl)

In [4]:
query = """
WITH current_and_previous as (
    select
        videoid,
        log_time,
        viewcount,
        lag(viewcount, 1) over (partition by videoid order by log_time) previous_value
    from video_data
)
select *,
    CAST(CASE 
        WHEN previous_value IS NULL THEN NULL
        ELSE ((viewcount - previous_value) / CAST(previous_value AS FLOAT)) * 100 
    END AS DECIMAL(10,2)) AS growth,
    case
        when viewcount > previous_value then '▲'
        when viewcount < previous_value then '▼'
        else '='
    end as trend
FROM TABLE(
    TUMBLE(TABLE current_and_previous, DESCRIPTOR(log_time), INTERVAL '1' MINUTE)
);
"""

t_results = t_env.execute_sql(query)

In [5]:
t_results.print()

+----+--------------------------------+-------------------------+----------------------+----------------------+-------------------------+-------------------------+-------------------------+--------------+--------------------------------+
| op |                        videoid |                log_time |            viewcount |       previous_value |            window_start |              window_end |             window_time |       growth |                          trend |
+----+--------------------------------+-------------------------+----------------------+----------------------+-------------------------+-------------------------+-------------------------+--------------+--------------------------------+
| +I |                    lFbany6g8Kw | 2025-02-13 18:24:28.983 |              3226726 |               <NULL> | 2025-02-13 18:24:00.000 | 2025-02-13 18:25:00.000 | 2025-02-13 18:24:28.983 |       <NULL> |                              = |
| +I |                    lFbany6g8Kw | 2025-02-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/gerardomartinez/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 